## Как работать с парсером hh вакансий


## Необходимо внести входные данные для парсера

1. Вносим название Google таблицы (например,Тарифы ежедневные) в строку `table name`

2. Вносим название листа, где вы будете хранить название вакансии и ссылки на них c hh  `links_sheet`(Например,в таблице "Сбор данных для презы hh2022, лист "Ссылки")
    - более того вы можете заранее установить необходимые фильтры, а после скопировать ссылку в Google, чтобы можно было выгружать вакансии с различными параметрами
    
3. указать диапозон ячеек где хранятся ссылки и названия, которые необходимо спарсить `links_range` 
    - диапозон обычный, например A1:B2 (у нас 3 блока для выгрузки на листе "Ссылки") **ВАЖНО: указывать с названиями столбцов.**
    
4. Указать название листа и папки, которые характеризуют выгрузку **c указанием даты выгрузки** `folder_name`
    - например. если мы хотим выгрузить Аналитиков, но внутри этой группы у нас разные названия: Аналитик данных, Аналитик SQL и т.п.
    - То есть это просто название листа в Google, в который будут грузиться ваши данные
    
5. Если вы хотите, чтобы в Google Таблице лист в выгрузкой создался автоматически, то укажите `create_new_sheet = True`, иначе создайте его сами и укажите `create_new_sheet = False`

6. Запустить скрипт путем `Cell` -> `Run All`


предварительно скачайте файл c доступом к API в Google Sheets - запросите у **Меринова Данилы**

файл загрузите в папку со скриптом, если планируете его переместить в другое место, то укажите путь до файла в `path_to_credential`


- **NOTE!** ``folder_name`` - это не только название листа, но и папка, которая создается на диске, в которую сохраняются вакансии, поэтому важно указать дату выгрузки, чтобы парсер мог проходиться по актуальным страницам в папке и **папки не дублировались**

- **NOTE_1!** путь до файла можно скопировать в проводнике, только заменить обратный слеш `\` на обычный `/`

- **NOTE_2!** если выдает ошибку, что библиотека не установлена, то надо зайти на **Home Page** в **Jupyter** -> нажать `New` -> `Terminal` - откроется окно с командной строкой. В ней необходимо установить библиотеки для этого пишем это в терминал: 

`pip install gspread`

`pip install oauth2client`

`pip install bs4`

`pip install selenium`


Если сбор данных будет производится в другую таблицу, то необходимо создать свзь с ней: необходимо предоставить доступ jupyter к гугл таблице, введя email бота (куда обычно вводит email)

Если лист в гугле уже есть, то пишем create_new_sheet = False

In [13]:
table_name = 'Сбор данных для презы hh2022'
links_sheet = 'Ссылки'

links_range = 'B240:C242'
folder_name = 'аналитики_джуны'
create_new_sheet = True

In [14]:
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import lxml
import os
import shutil
from tqdm import tqdm
from selenium.webdriver.chrome.service import Service

import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import date

In [15]:
# указать путь к credential
path_to_credential = 'path_to_credential' 

In [16]:
# Данные для доступа к Google Spreadsheets

# указать путь до credential
path_to_credential = path_to_credential 

# указать название таблицы, с которой будет работа
table_name = table_name

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_credential, scope)

gs = gspread.authorize(credentials)
work_table = gs.open(table_name)


In [17]:
def get_links(url, folder, add_folder=None, prof_obl=False):
    
    """
    функция проходит с помощью webdriver'a по всем ссылкам, считает кол-во страниц и на основании этого собирает
    все страницы с вакансиями для заданного запроса. После этого сохраняет страницы в папки
    
    url: str
        ссылка для запроса
    folder: str
        название папки
    user: str
        имя пользователя
    add_folder: str
        название дополнительной папки, если есть
    prof_obl: bool, default False
            True - если необходимо собирать данные из вкладки "Работа по профессиям"
    """
    
    options = webdriver.ChromeOptions()
    options.add_argument("user-agent=Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:84.0) Gecko/20100101 Firefox/84.0")
    options.add_argument("--disable-blink-features=AutomationControlled")

    s = Service(executable_path="chromedriver.exe")
    driver = webdriver.Chrome(service=s, options=options)
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'lxml')
        try:
            pages = int(soup.find_all(attrs={'class':'bloko-button','rel': 'nofollow', 'data-qa':'pager-page'})[-1].getText())
        except:
            pages = 1
        for page in range(pages):
            if prof_obl:
                driver.get(url+ '?page=' + str(page))
                time.sleep(0.25)
            else:
                driver.get(url+ '&page=' + str(page))
                time.sleep(0.25)
            
            if add_folder is not None:
                with open(f'{os.getcwd()}\\{add_folder}\\{folder}\\page_{page}_{date.today()}.html', 
                            "w", encoding='utf-8') as file:
                    file.write(driver.page_source)
            else:
                with open(f"{os.getcwd()}\\{folder}\\page_{page}_{date.today()}.html", 
                            "w", encoding='utf-8') as file:
                    file.write(driver.page_source)
    except Exception as ex:
                print(ex)

    finally:
        driver.close()
        driver.quit()

In [18]:
def get_list_of_vacancy(folder, add_folder=None):
    
    """
    вовзращает список с ссылками на на вакансии
    
    folder: str
        название папки
    user: str
        имя пользователя
    add_folder: str
        название дополнительной папки, если есть
    """
    
    links_list = []
    if add_folder is None:
        path = f"{os.getcwd()}\\{folder}"
    else:
        path = f"{os.getcwd()}\\{add_folder}/{folder}"
        
    for html in os.listdir(path):
        with open(f"{path}/{html}", encoding='utf-8') as file:
            src = file.read()

            try:
                soup = BeautifulSoup(src, "lxml")
                result = soup.find_all("a", class_="serp-item__title")
                if len(result) != 0:
                    for i in range(len(result)):
                        link = result[i].get('href')
                        links_list.append(link)
                else:
                    print(f'[INFO_PARSER] нет данных по вакансиям {html}')

            except:
                print('[INFO] проблема с парсингом')  
    return links_list

In [19]:
def get_vacancy_info(dict_with_links_list):
    
    """
    функция возвращает DataFrame с описанием вакансии
    
    dict_with_links_list: dict
        словарь, в котором ключ - папка, а значение список ссылок на вакансии по заданному ключу
    """
    
    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
    
    dict_with_df = {}
    
    for key in dict_with_links_list:
        name = []
        salary_from = []
        salary_to = []
        salary_currency = []
        salary_gross = []
        experience_name = []
        schedule_name = []
        employment_name = []
        description = []
        key_skills = []
        employer = []
        published_at = []
        url = []
        views = []
        
        for link in tqdm(dict_with_links_list[key]):
            flag = False
            retry = 0
            while flag == False:
                r = requests.get(link, headers=headers).text
                soup = BeautifulSoup(r, 'lxml')
                if soup.find(attrs={'data-qa': 'vacancy-title'}) is not None:
                    
                    # название вакансии
                    title = soup.find(attrs={'data-qa': 'vacancy-title'})
                    if title is not None:
                        name.append(title.getText())
                    else:
                        name.append('null')

                    # зарплата
                    salary = soup.find(attrs={'data-qa': 'vacancy-salary'})
                    if salary is not None:
                        salary = salary.getText().replace(u'\xa0', u'').split(' ')
                        if 'от' in salary:
                            salary_from_i = salary.index('от') + 1
                            salary_from.append(salary[salary_from_i])
                        else:
                            salary_from.append('null')
                        if 'до' in salary:
                            salary_to_i = salary.index('до') + 1
                            if salary[salary_to_i] == 'вычета':
                                salary_to.append('null')
                            else:
                                salary_to.append(salary[salary_to_i])
                        else:
                            salary_to.append('null')
                    else:
                        salary_from.append('null')
                        salary_to.append('null')

                    # валюта
                    sal_currency = soup.find(attrs={'data-qa': 'vacancy-salary'})
                    if sal_currency is not None:
                        sal_currency = sal_currency.getText().split(' ')
                        if 'руб.' in sal_currency:
                            currency = 'RUB'
                        elif 'USD' in sal_currency:
                            currency = 'USD'
                        elif 'EUR' in sal_currency:
                            currency = 'EUR'
                        elif 'KZT' in sal_currency:
                            currency = 'KZT'
                        elif 'бел. руб' in sal_currency:
                            currency = 'BYN'
                        elif 'KGS' in sal_currency:
                            currency = 'KGS'
                        elif 'сум' in sal_currency:
                            currency = 'UZS'
                        elif 'AZN' in sal_currency:
                            currency = 'AZN'
                        else:
                            currency = 'null'
                        salary_currency.append(currency)
                    else:
                        salary_currency.append('null')

                    # тип зарплаты
                    salary_type = soup.find('span', attrs={'class': 'vacancy-salary-compensation-type'})
                    if salary_type is not None:
                        salary_gross.append(salary_type.getText().strip())
                    else: 
                            salary_gross.append('null')

                    # опыт работы
                    experience = soup.find(attrs={'data-qa': 'vacancy-experience'})
                    if experience is not None:
                        experience_name.append(experience.getText())
                    else: 
                        experience_name.append('null')

                    # тип занятости, график работы
                    employment_schedule = soup.find(attrs={'data-qa': 'vacancy-view-employment-mode'})
                    if employment_schedule is not None:
                        employment_name.append(employment_schedule.getText().split(', ')[0])
                        schedule_name.append(employment_schedule.getText().split(', ')[1])
                    else: 
                        employment_name.append('null')
                        schedule_name.append('null')

                    # описание вакансии
                    descrip = soup.find('div', class_='vacancy-section')
                    if descrip is not None:
                        description.append(descrip.getText())
                    else:
                        description.append('null')

                    # ключевые навыки
                    key_skills_i = []
                    skills = soup.find_all(attrs={'class': 'bloko-tag bloko-tag_inline'})
                    if len(skills) != 0:
                        for i in soup.find_all(attrs={'class': 'bloko-tag bloko-tag_inline'}):
                            key_skills_i.append(i.getText())
                        key_skills.append(key_skills_i)
                    else:
                        key_skills.append('null')

                    # название компании
                    emp = soup.find('span', class_='vacancy-company-name')
                    if emp is not None:
                        employer.append(emp.getText())
                    else:
                        employer.append('null')

                    # кол-во просмотров сейчас
                    view = soup.find('span', class_='vacancy-viewers-count')
                    if view is not None:
                        views.append(view.getText().replace(u'\xa0', u' ').split(' ')[0])
                    else:
                        views.append('null')

                    # дата публикации
                    dt = soup.find('p', class_='vacancy-creation-time-redesigned')
                    if dt is not None:
                        published_at.append(dt.getText().replace(u'\xa0', u' ')\
                                                    .split('опубликована')[1]\
                                                    .split('в')[0].strip())
                    else:
                        published_at.append('null')

                    url.append(link)

                    time.sleep(0.25)

                    
                    flag = True

                    
                retry += 1
                if retry == 5:
                    break
                    
        output = pd.DataFrame()
        output['name'] = name
        output['salary_from'] = salary_from
        output['salary_to'] = salary_to
        output['salary_currency'] = salary_currency
        output['salary_gross'] = salary_gross
        output['experience_name'] = experience_name
        output['schedule_name'] = schedule_name
        output['employment_name'] = employment_name
        output['description'] = description
        output['key_skills'] = key_skills
        output['employer'] = employer
        output['published_at'] = published_at
        output['url'] = url
        output['views'] = views
        output['type'] = key
        
        dict_with_df[key] = output

    return dict_with_df

In [20]:
def count_skills(column, to_df=True):
    
    """
    функция возвращает словарь: ключ - навык, значение - кол-во позиций в выгрузке
    
    column: str
        столбец в датафрейме, который содержит ключевые навыки
    to_df: bool, default True
        возвращает датафрейм, если True, иначе - словарь
    """
    
    dct = {}
    for skill_list in column:
        for skill in skill_list:
            if skill not in dct.keys():
                dct[skill] = 1
            elif skill in dct.keys():
                dct[skill] += 1
    if to_df:
        df_skills = pd.DataFrame.from_dict(dct, orient='index')\
            .rename(columns={0:'num_skills'})\
            .sort_values('num_skills', ascending=False)
        return df_skills
    else:
        return dct

In [33]:
def upload_df(df, google_sheet, create_sheet=False, skill_df=False):
    
    """
    функция загружает df в Google Spreadsheets
    
    df: pandas.DataFrame
        DataFrame, который необходимо загрузить
    google_sheet: str
        навзание листа в Google Spreadsheets
    create_sheet: bool, default False
        создает новый лист, если True, иначе загружает данные в текущий лист
            
    """
    if skill_df:
        title = google_sheet + '_навыки'
    else:
        title = google_sheet
        
        df.key_skills = df.key_skills.astype('str')
        df.description = df.description.str.replace(u'\xa0', u' ').str.strip().str.replace(u'\n', u' ')
    
    if create_sheet:
        ws = work_table.add_worksheet(title=title, rows=100, cols=20)
    else:
        pass
    
    worksheet = work_table.worksheet(title)
    worksheet.append_row(df.columns.tolist(), value_input_option='USER_ENTERED')
    worksheet.append_rows(df.values.tolist(), value_input_option='USER_ENTERED')
    
    return print (f'Данные загружены на лист {title}')

# Пример выгрузки вакансий

In [22]:
# Получаем список ссылок
worksheet = work_table.worksheet(links_sheet)
data = worksheet.get_values(links_range)
columns = data.pop(0)
data = pd.DataFrame(data, columns=columns)

# создаем из них словарь
vacancy_links = dict(zip(data['название'].values, data['ссылки'].values))

# название папки для выгрузки
add_folder = folder_name
    
    
# создаем директорию внутри главной папки для выгрузок, если ее нет
try:
    # создаем директорию
    os.mkdir(f'{os.getcwd()}\\{add_folder}/')
    print("Directory " , add_folder ,  " Created ") 
except FileExistsError:
    print("Directory " , add_folder ,  " already exists")

    
# создаем папки для страниц, если таких папок еще не было
for new_folder in data['название'].values:
    try:
        # создаем директорию
        os.mkdir(f'{os.getcwd()}\\{add_folder}/{new_folder}')
        print("Directory " , new_folder ,  " Created ") 
    except FileExistsError:
        print("Directory " , new_folder ,  " already exists")
        
        
# проходим по всем ключам словаря (папка - url) и вызываем функцию
for key in vacancy_links:
    get_links(vacancy_links[key], key, add_folder=add_folder)
    
    
# создаем новый словарь, в котором ключ - папка, а значение список ссылок на вакансии по заданному ключу
vacancy_type_dict = {}
for key in vacancy_links:
    vacancy_type_dict[key] = get_list_of_vacancy(key, add_folder=add_folder)
    
    
all_df = get_vacancy_info(vacancy_type_dict)


# удаляем папку со скачанными страницами
shutil.rmtree(f'{os.getcwd()}\\{add_folder}')

Directory  аналитики_джуны  Created 
Directory  junior data analyst  Created 
Directory  junior data scientist  Created 


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:19<00:00,  1.04it/s]


In [23]:
# объединяем полученные датафреймы
best_df = pd.DataFrame(columns=['name', 'salary_from', 'salary_to', 'salary_currency', 'salary_gross',
       'experience_name', 'schedule_name', 'employment_name', 'description',
       'key_skills', 'employer', 'published_at', 'url', 'views', 'type'])

for key in all_df:
    best_df = pd.concat([best_df, all_df[key]]).reset_index(drop=True)

In [24]:
best_df

,name,salary_from,salary_to,salary_currency,salary_gross,experience_name,schedule_name,employment_name,description,key_skills,employer,published_at,url,views,type
0,Junior Data scientist (NLP),100000,150000,RUB,на руки,1–3 года,полный день,Полная занятость,\n\n\n\n\n\n\nПлатформа ОФД (аккредитованная И...,null,Платформа ОФД,21 марта 2023,https://pushkino.hh.ru/vacancy/77502259?from=v...,15,junior data analyst
1,Junior Data Analyst,null,null,null,null,не требуется,полный день,Полная занятость,О нас. Компания РЕСО-гарантия – один из лидеро...,null,"РЕСО-Гарантия, САО",20 марта 2023,https://pushkino.hh.ru/vacancy/78266507?from=v...,26,junior data analyst
2,Junior Data Engineer,null,null,null,null,1–3 года,полный день,Полная занятость,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nНаша команда з...,null,Сбер для экспертов,22 марта 2023,https://pushkino.hh.ru/vacancy/78397923?from=v...,23,junior data analyst
3,Junior Data Analyst,null,null,null,null,не требуется,полный день,Полная занятость,\n\n\n\n\nКарьера в АТОН\n\n\n\nАТОН - это ста...,[Python],ООО АТОН,16 марта 2023,https://pushkino.hh.ru/vacancy/78155321?from=v...,14,junior data analyst
4,Junior Data Engineer/ BI Аналитик,null,null,null,null,3–6 лет,полный день,Полная занятость,НПКЦ диагностики и телемедицины («Радиология М...,"[Английский язык, Научные исследования, Базы д...",ГБУЗ «Научно-практический клинический центр ди...,21 марта 2023,https://pushkino.hh.ru/vacancy/78371688?from=v...,7,junior data analyst
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Junior Data Scientist/Младший менеджер по обра...,null,null,null,null,1–3 года,полный день,Полная занятость,"Российский медийный холдинг ""Group4Media"" ищет...","[SQL, Python, Internet, Data Mining, MS PowerP...","Группа компаний «Group4Media», Управляющая ком...",1 марта 2023,https://pushkino.hh.ru/vacancy/77557573?from=v...,11,junior data scientist
72,Senior/ Middle data scientist (жизненный цикл ...,null,null,null,null,1–3 года,полный день,Полная занятость,\n\n\n\n\n\n\n\n\n\n\n\n ...,"[Python, SQL, Spark, Hive, Airflow, ML, CLTV]","ПАО ВТБ, Технологический блок",13 марта 2023,https://pushkino.hh.ru/vacancy/77986075?from=v...,4,junior data scientist
73,Senior Data Scientist в направление рекламной ...,null,null,null,null,3–6 лет,полный день,Полная занятость,"\n\n\n\n\n\n\nBig Data МТС – место, где телеко...","[Python, SQL, Математическая статистика, Big D...","ПАО «МТС», IT",5 марта 2023,https://pushkino.hh.ru/vacancy/74317205?from=v...,8,junior data scientist
74,French speaking recruitment intern - Стажер ре...,20000,50000,RUB,на руки,не требуется,полный день,Полная занятость,MP Recruiting is a fast growing independent Ex...,"[Французский язык, Research, Time management, ...",MP Recruiting,28 фе,https://pushkino.hh.ru/vacancy/77483584?from=v...,5,junior data scientist


In [25]:
# считаем ключевые навыки
skills = pd.DataFrame()

for key in all_df:
    t = count_skills(all_df[key].key_skills).assign(type = key)
    skills = pd.concat([t, skills])
    
skills = skills.drop(index=['n', 'u', 'l'])
skills = skills.reset_index()
skills = skills.rename(columns={'index':'skill'})


In [32]:
skills

,skill,num_skills,type
0,SQL,12,junior data scientist
1,Python,12,junior data scientist
2,Big Data,5,junior data scientist
3,Machine Learning,5,junior data scientist
4,ML,3,junior data scientist
...,...,...,...
192,Snowplow,1,junior data analyst
193,Анализ временных рядов,1,junior data analyst
194,A/B тесты,1,junior data analyst
195,BigQuery,1,junior data analyst


In [26]:
upload_df(best_df, folder_name, create_sheet=True)

Данные загружены на лист аналитики_джуны


In [34]:
upload_df(skills, folder_name, create_sheet=True, skill_df=True)

Данные загружены на лист аналитики_джуны_навыки
